# Day 7 - Handy Haversacks
## Investigation Notebook

This one is _tough_. I'm going to be writing up a solution (hopefully), but I'm probably going to have to lean on the subreddit a bit for when I inevitably get stuck.

### Part One - Approach

We need a way to load in the data and use regex. All lines have the following format:

```
x bags contain n_1 y bag, n_2 z bags.
```

I'm not going to be able to do this today, but I think we can load in the data using some regex. After that, I want to try and work through someone else's solution using the `networkx` python library.

In [144]:
import re
import networkx as nx
import networkx.algorithms as nxa

In [5]:
import re
import collections

In [146]:
# Initialise the network/graph
G = nx.DiGraph()

# Go through the input line by line
for line in lines:
    # Find nodes (n) and vertex/edges (v)
    n = re.search(NODE_REGEX, line)
    v = re.findall(VERTEX_REGEX, line)

    # if the node doesn't exist in the graph yet, add it
    if n[0] not in G.nodes:
        G.add_node(n[0])

    # if the bag can contain other bags, then add each one as an edge
    # add the number of bags as a weight to the graph
    if len(v) > 0:
        for each in v:
            inner_bag = each[1]
            w = int(each[0])
            if inner_bag not in G.nodes:
                # add the node if it doesn't exist
                G.add_node(inner_bag)

            # add the edge to the graph (syntax: edge, node, weight)
            G.add_edge(inner_bag, n[0], weight=w)

We're now going to do a _depth-first search_ of the graph to see where/how often the gold bag shows up. From my understanding, it should return the total number of nodes that connect to the shiny gold bag (including itself).

In [147]:
nodes = nxa.dfs_tree(G, "shiny gold").nodes

nodes = len(nodes) - 1
print(nodes)


142


From someone else's solution:

https://github.com/alexander-yu/adventofcode/blob/master/problems_2020/7.py

```
def get_size(graph, bag):
    return sum([
        graph.edges[bag, component]['weight'] * (1 + get_size(graph, component))
        for component in graph.successors(bag)
    ])
```



In [148]:
[n for n in G.successors('shiny gold')]

[int(G.edges['shiny gold', component]['weight']) for component in G.successors('shiny gold')]

[5, 4, 2, 4, 3, 1]

In [152]:
def get_size(graph, bag):
    return sum([
        int(graph.edges[bag, component]['weight']) * (1 + get_size(graph, component))
        for component in graph.successors(bag)
    ])

# REMEMBER TO REVERSE THE NETWORK - NOT IN PROVIDED SOLUTION
get_size(G.reverse(), 'shiny gold')

10219

That doesn't seem to want to work, so we try another solution:

```
H = G.reverse()
def get_sum(H, node):
    return sum(G[n][node]['weight'] * get_sum(H, n) for n in H.neighbors(node)) + 1

# part2
print(get_sum(H, "shinygold") - 1)
```


In [150]:
H = G.reverse()

def get_sum(H, node):
    return sum(H[node][n]['weight'] * get_sum(H, n) for n in H.neighbors(node)) + 1

get_sum(H, 'shiny gold') - 1

10219

In [145]:
with open("../data/day-7.txt", "r") as fp:
    lines=fp.readlines()

# Regex to extract the first bag
NODE_REGEX = r'(^\w+ \w+)'
m = re.search(NODE_REGEX, lines[0])
m[0]

# Regex to extract all other bags
VERTEX_REGEX = r'(\d+) (\w+ \w+)'
v = re.findall(VERTEX_REGEX, lines[0])